# Solutions

1. [Groupby Aggregation Basics](#1.-Groupby-Aggregation-Basics)
1. [Grouping and Aggregating with Multiple Columns](#2.-Grouping-and-Aggregating-with-Multiple-Columns)
1. [Grouping with Pivot Tables](#3.-Grouping-with-Pivot-Tables)
1. [Counting with Crosstabs](#4.-Counting-with-Crosstabs)
1. [Reshaping More](#5.-Reshaping-More)
1. [Create your own Data Analysis](#6.-Create-your-own-Data-Analysis)
1. [Alternate Groupby Syntax](#7.-Alternate-Groupby-Syntax)
1. [Custom Aggregation](#8.-Custom-Aggregation)
1. [Transform and Filter with Groupby](#9.-Transform-and-Filter-with-Groupby)

## 1. Groupby Aggregation Basics

In [1]:
import pandas as pd
emp = pd.read_csv('../data/employee.csv')
emp.head(3)

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black


### Exercise 1

<span  style="color:green; font-size:16px">Find the maximum salary for each sex.</span>

In [2]:
emp.groupby('sex').agg(max_salary=('salary', 'max'))

,max_salary
sex,
Female,342784.0
Male,342784.0


### Exercise 2

<span  style="color:green; font-size:16px">Find the median salary for each department.</span>

In [3]:
emp.groupby('dept').agg(median_salary=('salary', 'median')).head()

,median_salary
dept,
Fire,61921.08
Health & Human Services,50773.00
Houston Airport System,44200.00
Houston Public Works,46841.50
Library,34611.00


### Exercise 3

<span  style="color:green; font-size:16px">Find the average salary for each race. Return a DataFrame with the race as a column.</span>

In [4]:
emp.groupby('race').agg(avg_salary=('salary', 'mean')).round(-3).reset_index()

,race,avg_salary
0,Asian,65000.0
1,Black,52000.0
2,Hispanic,55000.0
3,Native American,58000.0
4,White,67000.0


Execute the cell below to read in the NYC deaths dataset and use it to answer the following exercises.

In [5]:
deaths = pd.read_csv('../data/nyc_deaths.csv')
deaths.head(3)

,year,cause,sex,race,deaths
0,2007,Accidents,F,Asian,32
1,2007,Accidents,F,Black,87
2,2007,Accidents,F,Hispanic,71


### Exercise 4

<span  style="color:green; font-size:16px">What year had the most deaths?</span>

In [6]:
year_deaths = deaths.groupby('year').agg(total=('deaths', 'sum'))
year_deaths

,total
year,
2007,53996
2008,54138
2009,52820
2010,52505
2011,52726
2012,52420
2013,53387
2014,53006


In [7]:
year_deaths.agg(['max', 'idxmax'])

,total
max,54138
idxmax,2008


### Exercise 5

<span  style="color:green; font-size:16px">Find the total number of deaths by race and sort by most to least.</span>

In [8]:
deaths.groupby('race').agg(total=('deaths', 'sum')).sort_values('total', ascending=False)

,total
race,
White,206487
Black,111116
Hispanic,74802
Asian,26355
Unknown,6238


## 2. Grouping and Aggregating with Multiple Columns

Execute the following cell to read in the City of Houston employee data and use it for the first few exercises.

In [9]:
import pandas as pd
emp = pd.read_csv('../data/employee.csv', parse_dates=['hire_date'])
emp.head(3)

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black


### Exercise 1

<span  style="color:green; font-size:16px">For each department and sex find the number of unique position titles, the total number of employees and the average salary. Make sure there is no multi-index for the index or columns.</span>

In [10]:
data = emp.groupby(['dept', 'sex']).agg(num_unique_titles=('title', 'nunique'),
                                        num_employees=('title', 'size'),
                                        avg_salaray=('salary', 'mean')).reset_index()
data.head(10)

,dept,sex,num_unique_titles,num_employees,avg_salaray
0,Fire,Female,51,240,62212.637250
1,Fire,Male,54,4136,60479.306862
2,Health & Human Services,Female,136,987,53838.310780
3,Health & Human Services,Male,110,366,59230.425956
4,Houston Airport System,Female,85,443,51099.300226
5,Houston Airport System,Male,113,773,57278.306598
6,Houston Public Works,Female,151,1195,51294.453004
7,Houston Public Works,Male,180,2995,51490.113309
8,Library,Female,55,404,41126.962921
9,Library,Male,44,159,44399.943396


### Exercise 2

<span  style="color:green; font-size:16px">For each department, race and sex find the min and max and salaries.</span>

In [11]:
emp.groupby(['dept','race', 'sex']).agg(min_salary=('salary', 'min'),
                                          max_salary=('salary', 'max')).head(10)

min_salary  max_salary
dept race            sex                           
Fire Asian           Female     39104.0   342784.00
                     Male       28024.0   342784.00
     Black           Female     16411.0   342784.00
                     Male       28024.0   342784.00
     Hispanic        Female     28024.0    89590.02
                     Male       26000.0   342784.00
     Native American Female     48189.7    70181.28
                     Male       28024.0   115835.98
     White           Female     16910.0   342784.00
                     Male       16515.0   342784.00

Execute the following cell to read in the college dataset and use it for the remaining exercises.

In [12]:
college = pd.read_csv('../data/college.csv')
college.head(3)

,instnm,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,...,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370


### Exercise 3
<span  style="color:green; font-size:16px">Which city name appears the most frequently. Do this in two different ways. Do it once with and once without the `groupby` method?</span>

In [13]:
size = college.groupby('city').agg(size=('stabbr', 'size'))
size.head()

,size
city,
Aberdeen,3
Abilene,5
Abingdon,2
Abington,1
Ada,3


In [14]:
size.sort_values('size', ascending=False).head()

,size
city,
New York,87
Chicago,78
Houston,72
Los Angeles,56
Miami,51


Can also just `size` directly and sort the series.

### Without groupby

Use `value_counts`

In [15]:
college['city'].value_counts().head()

New York       87
Chicago        78
Houston        72
Los Angeles    56
Miami          51
Name: city, dtype: int64

### Exercise 4

<span  style="color:green; font-size:16px">Does the city 'Houston' only appear in the state of Texas (abbreviated 'TX')?</span>

NO! It also appears in Missouri.

In [16]:
filt = college['city'] == 'Houston'
college.loc[filt, 'stabbr'].unique()

array(['TX', 'MO'], dtype=object)

Can see exact counts

In [17]:
college.loc[filt, 'stabbr'].value_counts()

TX    71
MO     1
Name: stabbr, dtype: int64

You can use a groupby and find the number of unique states for each city. This is not very efficient.

In [18]:
city_unique_state = college.groupby('city').agg(num_unique_states=('stabbr', 'nunique'))
city_unique_state.head()

,num_unique_states
city,
Aberdeen,2
Abilene,1
Abingdon,1
Abington,1
Ada,2


In [19]:
city_unique_state.loc['Houston']

num_unique_states    2
Name: Houston, dtype: int64

Also with `drop_duplicates`

In [20]:
college[['city', 'stabbr']].query('city == "Houston"') \
                           .drop_duplicates(subset='stabbr')

,city,stabbr
3617,Houston,TX
5366,Houston,MO


### Exercise 5
<span  style="color:green; font-size:16px">Find the maximum undergraduate population for each state?</span>

In [21]:
college.groupby('stabbr').agg(max_ugds=('ugds', 'max')).head(10)

,max_ugds
stabbr,
AK,12865.0
AL,29851.0
AR,21405.0
AS,1276.0
AZ,151558.0
CA,44744.0
CO,25873.0
CT,18016.0
DC,10433.0


### Exercise 6
<span  style="color:green; font-size:16px">Among colleges that have the largest undergrad population for each state, what is the difference between the most and least populous college?</span>

In [22]:
largest_per_state = college.groupby('stabbr').agg(max_ugds=('ugds', 'max'))
largest_per_state.max() - largest_per_state.min()

max_ugds    150956.0
dtype: float64

### Exercise 7
<span  style="color:green; font-size:16px">Find the name and population of the largest college per state.</span>

In [23]:
# set the index first to be instnm so that you can take advantage of idxmax
c2 = college.set_index('instnm')
max_indexes = c2.groupby('stabbr').agg(max_ugds=('ugds', 'max'),
                                       max_ugds_college=('ugds', 'idxmax'))
max_indexes.head(10)

,max_ugds,max_ugds_college
stabbr,,
AK,12865.0,University of Alaska Anchorage
AL,29851.0,The University of Alabama
AR,21405.0,University of Arkansas
AS,1276.0,American Samoa Community College
AZ,151558.0,University of Phoenix-Arizona
CA,44744.0,Ashford University
CO,25873.0,University of Colorado Boulder
CT,18016.0,University of Connecticut
DC,10433.0,George Washington University


### Another way
Use the **`first`** groupby method to return the first row of each group after sorting.

In [24]:
cols = ['stabbr', 'instnm', 'ugds']
college_trim = college[cols]

# sort by state then by population descending
college_trim_sort = college_trim.sort_values(['stabbr', 'ugds'], ascending=[True, False])


# group by state and take the first in the group
college_trim_sort.groupby('stabbr').first().head()

,instnm,ugds
stabbr,,
AK,University of Alaska Anchorage,12865.0
AL,The University of Alabama,29851.0
AR,University of Arkansas,21405.0
AS,American Samoa Community College,1276.0
AZ,University of Phoenix-Arizona,151558.0


### Use `sort_values` with `drop_duplicates`
We've done this in previous notebooks. No grouping.

In [25]:
college_trim.sort_values(['stabbr', 'ugds'], ascending=[True, False]) \
            .drop_duplicates(subset='stabbr').head(10)

,stabbr,instnm,ugds
60,AK,University of Alaska Anchorage,12865.0
5,AL,The University of Alabama,29851.0
137,AR,University of Arkansas,21405.0
4138,AS,American Samoa Community College,1276.0
7116,AZ,University of Phoenix-Arizona,151558.0
1299,CA,Ashford University,44744.0
574,CO,University of Colorado Boulder,25873.0
641,CT,University of Connecticut,18016.0
701,DC,George Washington University,10433.0
691,DE,University of Delaware,18222.0


### Exercise 8
<span  style="color:green; font-size:16px">Do distance only schools tend to have more or less student population than non-distance-only schools?</span>

In [26]:
# They have more
college.groupby('distanceonly').agg(mean_ugds=('ugds', 'mean'))

,mean_ugds
distanceonly,
0.0,2334.648135
1.0,6245.743590


### Exercise 9
<span  style="color:green; font-size:16px">Do distance only schools tend to be more or less religously affiliated than non-distance-only schools?</span>

In [27]:
# Less
college.groupby('distanceonly').agg(mean_relaffil=('relaffil', 'mean'))

,mean_relaffil
distanceonly,
0.0,0.149635
1.0,0.050000


### Exercise 10
<span  style="color:green; font-size:16px">What state has the lowest percentage of currently operating schools of those that have religious affiliation?</span>

In [28]:
filt = college['relaffil'] == 1
cr = college[filt]
rel_oper_mean = cr.groupby('stabbr').agg(mean_curroper=('curroper', 'mean'))
rel_oper_mean.head()

,mean_curroper
stabbr,
AK,1.000000
AL,0.916667
AR,0.944444
AZ,0.444444
CA,0.585366


In [29]:
rel_oper_mean.sort_values('mean_curroper').head()

,mean_curroper
stabbr,
UT,0.400000
AZ,0.444444
NV,0.500000
CA,0.585366
CT,0.647059


### Exercise 11

<span  style="color:green; font-size:16px">Find the top 5 historically black colleges that have the highest undergraduate white percentage (ugds_white)?</span>

In [30]:
filt = college['hbcu'] == 1
cols = ['instnm', 'ugds_white']
college.loc[filt, cols].sort_values('ugds_white', ascending=False).head()

,instnm,ugds_white
4021,Bluefield State College,0.8437
17,Gadsden State Community College,0.6921
4050,West Virginia State University,0.5816
48,Shelton State Community College,0.5613
55,H Councill Trenholm State Community College,0.3951


## 3. Grouping with Pivot Tables

In [31]:
import pandas as pd
flights = pd.read_csv('../data/flights.csv', parse_dates=['date'])
flights['day_of_week'] = flights['date'].dt.day_name()
flights['month'] = flights['date'].dt.month_name()
flights.head(3)

,date,airline,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,day_of_week,month
0,2018-01-01,UA,LAS,IAH,100,547,0,134.0,1222.0,0,0,0,0,0,Monday,January
1,2018-01-01,B6,PSE,MCO,500,707,0,170.0,1179.0,0,0,9,0,138,Monday,January
2,2018-01-01,WN,DEN,PHX,515,720,0,91.0,602.0,0,0,0,0,0,Monday,January


In [32]:
flights.shape

(300000, 16)

### Exercise 1
<span  style="color:green; font-size:16px">What is the carrier departure delay for each day of the week for each airline? Highlight the worst day of the week for each airline.</span>

In [33]:
avg_delay = flights.pivot_table(index='airline', columns='day_of_week', 
                                values='carrier_delay').round(1)
avg_delay.style.highlight_max(axis='columns')

day_of_week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
airline,,,,,,,
9E,4.9,4.2,2.8,2.5,4.3,3.2,3
AA,4.6,3.9,4.1,4.4,4.8,3.9,3.5
AS,2,2.2,2.3,2.3,2.2,1.5,2.3
B6,7.3,5.2,6,5.6,5.7,6.1,4.8
DL,3.4,2.9,3.2,3.3,3.3,3.1,3
EV,4.8,4.9,6,6.4,4.7,4.6,3.8
F9,5.5,4.1,8.2,6,6.3,5.2,3.4
G4,5.9,5.5,6.6,5.4,4.9,5.3,4.3
HA,2.1,2.5,2.8,3,2,1.9,1.7


You can highlight min and max by chaining style methods.

In [34]:
avg_delay.style.highlight_max(axis='columns') \
         .highlight_min(axis='columns', color='lightblue')

day_of_week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
airline,,,,,,,
9E,4.9,4.2,2.8,2.5,4.3,3.2,3
AA,4.6,3.9,4.1,4.4,4.8,3.9,3.5
AS,2,2.2,2.3,2.3,2.2,1.5,2.3
B6,7.3,5.2,6,5.6,5.7,6.1,4.8
DL,3.4,2.9,3.2,3.3,3.3,3.1,3
EV,4.8,4.9,6,6.4,4.7,4.6,3.8
F9,5.5,4.1,8.2,6,6.3,5.2,3.4
G4,5.9,5.5,6.6,5.4,4.9,5.3,4.3
HA,2.1,2.5,2.8,3,2,1.9,1.7


### Exercise 2

<span  style="color:green; font-size:16px">Use a `pivot_table` to find the total number of canceled flights for each origin airport and airline making sure the airline is in the columns. Use the result to find the origin airport with the most cancelled flights for each airline. Also return this maximum number of cancelled flights.</span>

In [35]:
airline_cancel = flights.pivot_table(index='origin', columns='airline', 
                                     values='cancelled', aggfunc='sum', fill_value=0)
airline_cancel.head(10)

airline,9E,AA,AS,B6,DL,EV,F9,G4,HA,MQ,NK,OH,OO,UA,VX,WN,YV,YX
origin,,,,,,,,,,,,,,,,,,
ABE,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0
ABI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ABQ,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
ABR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ABY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ACK,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1
ACT,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0
ACV,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0
ACY,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [36]:
airline_cancel.agg(['max', 'idxmax'])

,9E,AA,AS,B6,DL,EV,F9,G4,HA,MQ,NK,OH,OO,UA,VX,WN,YV,YX
max,46,89,15,55,38,41,8,2,5,86,8,164,96,47,6,80,44,58
idxmax,LGA,DFW,SEA,BOS,ATL,EWR,AUS,AZA,HNL,ORD,EWR,CLT,ORD,EWR,LAX,BWI,DFW,LGA


### Exercise 3

<span  style="color:green; font-size:16px">Find the total distance flown for each airline for each month. Highlight the month with the most number of miles flown and use the style `format` method to put commas in the numbers so that they are easier to read.</span>

In [37]:
total_dist = flights.pivot_table(index='airline', columns='month', 
                                 values='distance', aggfunc='sum')

In [38]:
total_dist.style.format('{:,.0f}').highlight_max(axis='columns')

month,April,August,December,February,January,July,June,March,May,November,October,September
airline,,,,,,,,,,,,
9E,"396,909","450,367","380,262","356,824","352,039","417,118","436,066","393,495","438,168","389,146","433,166","375,398"
AA,"3,198,113","3,452,292","3,057,325","2,908,092","3,144,884","3,451,232","3,430,104","3,319,815","3,269,069","3,104,159","3,294,608","3,244,707"
AS,"1,206,809","1,249,375","1,159,489","717,196","778,335","1,335,399","1,275,782","795,520","1,267,618","1,135,590","1,167,996","1,143,964"
B6,"1,147,885","1,178,619","1,210,748","1,002,637","1,120,112","1,263,731","1,188,803","1,193,019","1,174,398","1,136,709","1,064,945","1,154,102"
DL,"2,987,292","3,231,739","2,822,366","2,432,165","2,560,300","3,381,699","3,084,996","3,019,979","3,082,591","2,706,367","2,882,438","2,838,155"
EV,"352,288","346,896","264,980","341,469","389,356","341,732","334,172","427,972","322,236","269,671","263,082","310,946"
F9,"402,585","474,373","457,704","402,441","410,437","444,052","429,664","391,607","432,664","436,044","419,548","437,945"
G4,"321,384","300,061","282,951","254,794","272,989","365,067","368,208","355,673","300,633","260,225","279,445","209,225"
HA,"169,034","191,050","210,183","186,465","165,274","228,420","222,633","240,452","179,908","169,517","202,204","188,017"


### Exercise 4
<span  style="color:green; font-size:16px">Use the City of Houston employee dataset for this exercise. You can create pivot tables with multiple columns in the index or the columns by using a list. Create a pivot table with the department as the index and the race and sex as the columns. Calculate the median salary for these cross sections.</span>

In [39]:
emp = pd.read_csv('../data/employee.csv')
emp.pivot_table(index='dept', columns=['race', 'sex'], 
                values='salary', aggfunc='median').round(-3).style.format('{:,.0f}')

## 4. Counting with Crosstabs

In [40]:
import pandas as pd
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200
mh = pd.read_csv('../data/mental_health.csv')
mh.head(3)

,timestamp,age,gender,country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,No,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,No,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,No,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN


### Exercise 1
<span  style="color:green; font-size:16px">Do people with a family history of mental illness seek treatment more often than those who do not?</span>

In [41]:
pd.crosstab(index=mh['family_history'], columns=mh['treatment'])

treatment,No,Yes
family_history,,
No,427,255
Yes,116,343


In [42]:
pd.crosstab(index=mh['family_history'], columns=mh['treatment'], normalize='index').round(2)

treatment,No,Yes
family_history,,
No,0.63,0.37
Yes,0.25,0.75


Yes, there is a large difference. 75% of people with a family history seek treatment vs 37% for those who have not.

### Exercise 2
<span  style="color:green; font-size:16px">Find the total number and ratio of employees that seek treatment for companies that provide health benefits vs those that do not.</span>

In [43]:
pd.crosstab(index=mh['benefits'], columns=mh['treatment'])

treatment,No,Yes
benefits,,
Don't know,231,141
No,146,158
Yes,166,299


In [44]:
pd.crosstab(index=mh['benefits'], columns=mh['treatment'], normalize='index').round(2)

treatment,No,Yes
benefits,,
Don't know,0.62,0.38
No,0.48,0.52
Yes,0.36,0.64


### Exercise 3
<span  style="color:green; font-size:16px">You can provide a list of multiple columns to both the `index` and `columns` parameters of the `crosstab` function. Put country and number of employees in the index and benefits and treatment in the columns. It's probably easier to make separate list variables first.</span>

In [45]:
index = [mh['country'], mh['no_employees']]
columns = [mh['benefits'], mh['treatment']]
pd.crosstab(index=index, columns=columns)

benefits                      Don't know      No     Yes    
treatment                             No Yes  No Yes  No Yes
country        no_employees                                 
Australia      1-5                     1   0   1   1   0   0
               100-500                 1   0   1   2   0   2
               26-100                  0   0   1   3   0   0
               500-1000                1   0   0   0   0   0
               6-25                    0   1   0   3   0   0
               More than 1000          1   0   0   0   1   1
Canada         1-5                     1   0   5   5   0   0
               100-500                 2   3   0   0   2   4
               26-100                  4   4   3   1   3   3
               500-1000                0   0   0   0   0   1
               6-25                    4   2   6   1   3   5
               More than 1000          0   2   0   0   2   6
France         100-500                 1   0   1   0   0   0
               26-100                  0   0   3   1   0   0
               500-1000                1   0   0   0   1   0
               6-25                    1   0   3   0   0   1
Germany        1-5                     0   0   3   3   0   1
               100-500                 1   3   1   0   0   0
               26-100                  2   4   4   3   1   0
               500-1000                1   0   0   0   1   0
               6-25                    5   1   3   3   0   2
               More than 1000          2   1   0   0   0   0
Ireland        1-5                     1   0   5   4   0   0
               100-500                 1   0   0   1   0   0
               26-100                  0   1   2   3   0   0
               500-1000                0   0   0   0   0   1
               6-25                    0   0   1   2   0   0
               More than 1000          2   0   1   0   1   1
Netherlands    1-5                     2   2   1   2   0   1
               100-500                 1   0   0   0   1   1
               26-100                  1   0   2   1   0   0
               500-1000                1   0   0   0   0   0
               6-25                    1   1   7   1   0   0
               More than 1000          0   0   0   0   1   0
United Kingdom 1-5                     6   3   8  13   0   1
               100-500                 5   6   2   4   2   1
               26-100                 12   5   7  11   3   2
               500-1000                2   3   1   0   1   0
               6-25                   11   8  23  14   1   1
               More than 1000          5   6   3   7   0   8
United States  1-5                    11   4  16  31   3  11
               100-500                22  15   3   8  23  42
               26-100                 42  27   7   7  21  66
               500-1000                9   1   1   1  10  20
               6-25                   32  23  16  20  17  26
               More than 1000         35  15   5   2  68  91

## 5. Alternate Groupby Syntax

## 6. Custom Aggregation

Execute the cell below to read in the flights dataset and then use it for the following exercises.

In [46]:
import pandas as pd
flights = pd.read_csv('../data/flights.csv', parse_dates=['date'])
flights.head(3)

,date,airline,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-01-01,UA,LAS,IAH,100,547,0,134.0,1222.0,0,0,0,0,0
1,2018-01-01,B6,PSE,MCO,500,707,0,170.0,1179.0,0,0,9,0,138
2,2018-01-01,WN,DEN,PHX,515,720,0,91.0,602.0,0,0,0,0,0


### Exercise 1
<span  style="color:green; font-size:16px">What are the 3 least common airlines?</span>

In [47]:
flights['airline'].value_counts().tail(3)

G4    4074
HA    3408
VX     710
Name: airline, dtype: int64

### Exercise 2
<span  style="color:green; font-size:16px">For each airline, find out what percentage of its flights leave on a Tuesday. Use a custom aggregation function.</span>

In [48]:
def tuesday_pct(s):
    return (s.dt.day_name() == 'Tuesday').mean()

flights.groupby('airline').agg(percent_tuesday=('date', tuesday_pct))

,percent_tuesday
airline,
9E,0.150379
AA,0.143346
AS,0.142011
B6,0.138477
DL,0.143696
EV,0.144190
F9,0.144545
G4,0.072410
HA,0.147300


### Exercise 3
<span  style="color:green; font-size:16px">Redo Exercise 2 without using a custom aggregation Exercise. What is the performance difference?</span>

In [49]:
flights['is_tuesday'] = flights['date'].dt.day_name() == 'Tuesday'
flights.groupby('airline').agg(percent_tuesday=('is_tuesday', 'mean'))

,percent_tuesday
airline,
9E,0.150379
AA,0.143346
AS,0.142011
B6,0.138477
DL,0.143696
EV,0.144190
F9,0.144545
G4,0.072410
HA,0.147300


About 50% improvement

In [50]:
%timeit -n 1 flights.groupby('airline').agg(percent_tuesday=('date', tuesday_pct))

119 ms ± 3.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%%timeit -n 1
# flights['is_tuesday'] = flights['date'].dt.day_name() == 'Tuesday'
flights.groupby('airline').agg(percent_tuesday=('is_tuesday', 'mean'))

20.3 ms ± 638 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Exercise 4
<span  style="color:green; font-size:16px">The range of undergrad populations per state was calculated using the `min_max` custom function from the top of this notebook. Use this same function to calculate the range of distance for each airline. Then calculate this range again without a custom function.</span>

In [52]:
def min_max(s):
    return s.max() - s.min()

In [53]:
flights.groupby('airline').agg(dist_range=('distance', min_max))

,dist_range
airline,
9E,1317.0
AA,4160.0
AS,2843.0
B6,2636.0
DL,4889.0
EV,1090.0
F9,2129.0
G4,1641.0
HA,4899.0


In [54]:
dist_min_max = flights.groupby('airline').agg(max_dist=('distance', 'max'),
                                              min_dist=('distance', 'min'))
dist_min_max['dist range'] = dist_min_max['max_dist'] - dist_min_max['min_dist']
dist_min_max

,max_dist,min_dist,dist range
airline,,,
9E,1391.0,74.0,1317.0
AA,4243.0,83.0,4160.0
AS,2874.0,31.0,2843.0
B6,2704.0,68.0,2636.0
DL,4983.0,94.0,4889.0
EV,1157.0,67.0,1090.0
F9,2446.0,317.0,2129.0
G4,1900.0,259.0,1641.0
HA,4983.0,84.0,4899.0


### Exercise 5
<span  style="color:green; font-size:16px">For each airline, return the first and last row of each group. Use one of the direct [Groupby methods][1]</span>

[1]: http://pandas.pydata.org/pandas-docs/stable/api.html#groupby

In [55]:
flights.groupby('airline').nth([0, -1]).head()

,date,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,is_tuesday
airline,,,,,,,,,,,,,,
9E,2018-01-01,BNA,MCO,700,1003,0,90.0,616.0,0,0,28,0,0,False
9E,2018-12-31,DTW,TVC,2045,2208,0,44.0,207.0,0,778,0,0,0,False
AA,2018-12-31,LAS,MIA,2322,650,0,244.0,2174.0,0,0,0,0,0,False
AA,2018-01-01,ORD,MIA,515,922,0,154.0,1197.0,0,0,0,0,0,False
AS,2018-12-31,SEA,DFW,2315,502,0,210.0,1660.0,3,0,3,0,26,False


## 9. Transform and Filter with Groupby

Execute the following cell to read in the college dataset and then use it for the following exercises.

In [56]:
pd.options.display.max_columns = 100
college = pd.read_csv('../data/college.csv')
college.head(3)

,instnm,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,ugds,ugds_white,ugds_black,ugds_hisp,ugds_asian,ugds_aian,ugds_nhpi,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370


### Exercise 1
<span  style="color:green; font-size:16px">Filter the college DataFrame for states that have more than 500,000 total undergraduate students. Can you verify your results?</span>

In [57]:
college_large = college.groupby('stabbr').filter(lambda sub_df: sub_df['ugds'].sum() > 500000)
college_large.head()

,instnm,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,ugds,ugds_white,ugds_black,ugds_hisp,ugds_asian,ugds_aian,ugds_nhpi,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
43,Prince Institute-Southeast,Elmhurst,IL,0.0,0.0,0.0,0,NaN,NaN,0.0,84.0,0.7976,0.1310,0.0714,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,1,0.7857,0.9375,0.6569,PrivacySuppressed,20992
68,Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,4102.0,0.3162,0.4405,0.0763,0.0017,0.0207,0.0046,0.0373,0.0,0.1026,0.4749,0,0.8291,0.7151,0.6700,28600,9500
69,Collins College,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,83.0,0.3253,0.0843,0.1566,0.0000,0.0241,0.0000,0.0241,0.0,0.3855,0.3373,0,0.7205,0.8228,0.4764,25700,47000
70,Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,25.0,0.7600,0.0400,0.1200,0.0000,0.0000,0.0400,0.0400,0.0,0.0000,0.1600,0,0.6349,0.5873,0.4651,17800,9588
71,Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,126.0,0.2143,0.0873,0.5794,0.0159,0.0873,0.0079,0.0000,0.0,0.0079,0.2222,1,0.7962,0.6615,0.4229,18200,9833


In [58]:
college_large.groupby('stabbr').agg(ugds_total=('ugds', 'sum')) \
             .sort_values('ugds_total', ascending=False).round(-3)

,ugds_total
stabbr,
CA,2304000.0
TX,1277000.0
NY,994000.0
FL,960000.0
PA,605000.0
IL,600000.0
OH,538000.0
AZ,520000.0


Execute the following cell to read in the City of Houston employee dataset and then use it for the following exercises.

In [59]:
emp = pd.read_csv('../data/employee.csv')
emp.head()

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black
3,Police,SENIOR POLICE OFFICER,1997-05-27,75942.10,Male,Hispanic
4,Police,SENIOR POLICE OFFICER,2006-01-23,69355.26,Male,White


### Exercise 2

<span  style="color:green; font-size:16px">Filter it so that only position titles with an average salary of 100,000 remain. Can you verify your results?</span>

In [60]:
high_sal = emp.groupby('title').filter(lambda sub_df: sub_df['salary'].mean() > 100000)
high_sal.head()

,dept,title,hire_date,salary,sex,race
16,Other,ASSOCIATE JUDGE OF MUNICIPAL COURTS,2005-11-09,107744.00,Male,Hispanic
17,Police,POLICE COMMANDER,1983-02-07,115821.42,Male,White
19,Other,ASSISTANT DIRECTOR (EXECUTIVE LEVEL),2002-05-28,95783.00,Female,Hispanic
39,Houston Airport System,DEPUTY ASSISTANT DIRECTOR (EXECUTIVE LEV,2017-08-15,112270.00,Male,Black
48,Fire,ASSISTANT FIRE CHIEF,1994-11-07,115835.98,Male,Hispanic


In [61]:
high_sal.groupby('title').agg(avg_salary=('salary', 'mean')).min()

avg_salary    100038.0
dtype: float64

### Exercise 3
<span  style="color:green; font-size:16px">Filter the employee dataset so that only position titles with at least 5 employees and an average salary of $80,000 remain. Can you verify the results?</span>

In [62]:
def sal_count(sub_df):
    return sub_df['salary'].mean() > 80000 and len(sub_df) >= 5

In [63]:
high_sal_count = emp.groupby('title').filter(sal_count)
high_sal_count.head()

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
16,Other,ASSOCIATE JUDGE OF MUNICIPAL COURTS,2005-11-09,107744.00,Male,Hispanic
17,Police,POLICE COMMANDER,1983-02-07,115821.42,Male,White
19,Other,ASSISTANT DIRECTOR (EXECUTIVE LEVEL),2002-05-28,95783.00,Female,Hispanic


In [64]:
high_sal_count.groupby('title').agg(avg_salary=('salary', 'mean'),
                                size=('salary', 'size')).min()

avg_salary    80153.202222
size              5.000000
dtype: float64

### Exercise 4

<span  style="color:green; font-size:16px">Add a new column, **pct_max_dept_sex**, to the employee DataFrame that holds the employees percentage of the maximum salary for each department and race. For instance, if a male HPD employee makes 80,000 and the maximum male HPD salary is 120,000 then the value for this employee would be 80,000/120,000 or .666. Verify this value for the first employee.</span>

In [65]:
def pct_max(sub_series):
    return sub_series / sub_series.max()

In [66]:
emp['pct_max_dept_sex'] = emp.groupby(['dept', 'sex']).transform(pct_max)
emp.head()

,dept,title,hire_date,salary,sex,race,pct_max_dept_sex
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White,0.312662
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic,0.298844
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black,0.227809
3,Police,SENIOR POLICE OFFICER,1997-05-27,75942.10,Male,Hispanic,0.271222
4,Police,SENIOR POLICE OFFICER,2006-01-23,69355.26,Male,White,0.247697


In [67]:
filt = (emp['dept'] == 'Police') & (emp['sex'] == 'Male')
max_sal = emp.loc[filt, 'salary'].max()
max_sal

280000.0

In [68]:
emp.loc[0, 'salary'] / max_sal

0.31266207142857144